In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# New York Stock Exchange average monthly returns [1961-1966] from curriculum
nyse = pd.read_csv("NYSE_monthly.csv")
col_name= 'Month'
nyse[col_name] = pd.to_datetime(nyse[col_name])
nyse.set_index(col_name, inplace=True)

# Types of Trends

## Stationary

### Definition:
> images from [https://www.analyticsvidhya.com/blog/2015/12/complete-tutorial-time-series-modeling/](https://www.analyticsvidhya.com/blog/2015/12/complete-tutorial-time-series-modeling/)

- The series' **mean** is **not** a function of time
![https://www.analyticsvidhya.com/wp-content/uploads/2015/02/Mean_nonstationary.png](images/Mean_nonstationary.png)

- The series' **varaince** is **not** a function of time (heteroscedacity)
![https://www.analyticsvidhya.com/wp-content/uploads/2015/02/Var_nonstationary.png](images/Var_nonstationary.png)

- The series' **covaraince** is **not** a function of time
![https://www.analyticsvidhya.com/wp-content/uploads/2015/02/Cov_nonstationary.png](images/Cov_nonstationary.png)

### No Trend

In [ ]:
data = nyse
data.plot(figsize=(12,6), linewidth=2, fontsize=14)
plt.xlabel(col_name, fontsize=20)
plt.ylabel("Monthly NYSE returns", fontsize=20)
plt.ylim((-0.15,0.15));

## Linear Trend

### Upward

![](https://github.com/learn-co-students/dsc-3-25-05-types-of-trends-online-ds-sp-000/raw/master/index_files/index_15_0.png)

### Downward

![](https://github.com/learn-co-students/dsc-3-25-05-types-of-trends-online-ds-sp-000/raw/master/index_files/index_19_0.png)

## Exponential

![](https://github.com/learn-co-students/dsc-3-25-05-types-of-trends-online-ds-sp-000/raw/master/index_files/index_22_0.png)

## Periodic

![](https://github.com/learn-co-students/dsc-3-25-05-types-of-trends-online-ds-sp-000/raw/master/index_files/index_25_0.png)

![](https://github.com/learn-co-students/dsc-3-25-05-types-of-trends-online-ds-sp-000/raw/master/index_files/index_30_0.png)

# Assessing Trends 

In [ ]:
# generated data 
years = pd.date_range('2012-01', periods=72, freq="M")
index = pd.DatetimeIndex(years)

np.random.seed(3456)
sales= np.random.randint(-4, high=4, size=72)
bigger = np.array([0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,
                   3,3,3,3,3,3,3,3,7,7,7,7,7,7,7,7,7,7,7,
                   11,11,11,11,11,11,11,11,11,11,18,18,18,
                   18,18,18,18,18,18,26,26,26,26,26,36,36,36,36,36])
data = pd.Series(sales+bigger+6, index=index)
ts=data
fig = plt.figure(figsize=(12,6))
plt.plot(data)
plt.xlabel("month", fontsize=16)
plt.ylabel("monthly sales", fontsize=16)
plt.show()

## Rolling Statistics

Take the average of a number of past data points (over a time period)

### Example

In [ ]:
rolmean = ts.rolling(window = 8, center = False).mean()
rolstd = ts.rolling(window = 8, center = False).std()

fig = plt.figure(figsize=(12,7))
orig = plt.plot(ts, color='blue',label='Original')
mean = plt.plot(rolmean, color='red', label='Rolling Mean')
std = plt.plot(rolstd, color='black', label = 'Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

## Dickey-Fuller Test

Statistical test for testing stationarity; $H_0$ is that time series is stationary

Doc Resource: http://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html

### Code Example

In [ ]:
from statsmodels.tsa.stattools import adfuller

dftest = adfuller(ts)

# Extract and display test results in a user friendly manner
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dftest)

In [ ]:
print(dfoutput)

# Removing Trends

## Log Transformation

Penalize higher values more (similar alternatives: square & cube roots)

- Clear & significant positive trend but maybe not linear
- Certain heteroskedasticity

## Subtract the Rolling Mean

## Differencing

# Time Series Decomposition